<a href="https://colab.research.google.com/github/samwong3333/try/blob/main/5_MCSimulation_20205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# change directory from google colab into google drive, connect to google drive first
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
# change the present working directory
import os
os.chdir('/content/drive/MyDrive/RMSC6007')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# feel free to modify, for example, change the context to "notebook"
sns.set_theme(context="talk", style="whitegrid",
              palette="colorblind", color_codes=True,
              rc={"figure.figsize": [12, 8]})

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Chapter 6: Monte Carlo simulations in finance

Monte Carlo simulations are a class of computational algorithms that use repeated random sampling to solve any problems that have a probabilistic interpretation. In finance, one of the reasons they gained popularity is that they can be used to accurately estimate integrals. The main idea of Monte Carlo simulations is to produce a multitude of sample paths—possible scenarios/outcomes, often over a given period of time. The horizon is then split into a specified number of time steps and the process of doing so is called **discretization**. Its goal is to approximate continuous time, since the pricing of financial instruments happens in continuous time.

The results from all these simulated sample paths can be used to calculate metrics such as the percentage of times an event occurred, the average value of an instrument at the last step, and so on. Historically, the main problem with the Monte Carlo approach was that it required heavy computational power to calculate all possible scenarios. Nowadays, it is becoming less of a problem as we can run fairly advanced simulations on a desktop computer or a laptop.

By the end of this chapter, we will have seen how we can use Monte Carlo methods in various scenarios and tasks. In some of them, we will create the simulations from scratch, while in others, we will use modern Python libraries to make the process even easier. Due to the method's flexibility, Monte Carlo is one of the most important techniques in computational finance. It can be adapted to various problems, such as pricing derivatives with no closed-form solution (American/Exotic options), valuating bonds (for example, a zero-coupon bond), estimating the uncertainty of a portfolio (for example, by calculating Value-at-Risk and Expected Shortfall), or carrying out stress-tests in risk management. We show how to solve some of these problems in this chapter.

In this chapter, we cover the following recipes:

* Simulating stock price dynamics using Geometric Brownian Motion
* Pricing European options using simulations
* Pricing American options with Least Squares Monte Carlo
* Pricing American options using Quantlib
* Estimating value-at-risk using Monte Carlo

See also: https://www.ibm.com/cloud/learn/monte-carlo-simulation

## Simulating stock price dynamics using Geometric Brownian Motion

Thanks to the unpredictability of financial markets, simulating stock prices plays an important role in the valuation of many derivatives, such as options. Due to the aforementioned randomness in price movement, these simulations rely on **stochastic differential equations (SDE)**.

A stochastic process is said to follow the **Geometric Brownian Motion (GBM)** when it satisfies the following SDE:

<center>$dS_t= \mu S_tdt + \sigma S_tdW_t $</center>

Here, we have the following:

* $S_t$: Stock price
* $\mu$: The drift coefficient, that is, the average return over a given period or the instantaneous expected return
* $\sigma$: The diffusion coefficient, that is, how much volatility is in the drift
* $W_t$: The Brownian Motion

We will not investigate the properties of the Brownian Motion in too much depth, as it is outside the scope of this book. Suffice to say, Brownian increments are calculated as a product of a Standard Normal random variable $(rv ∼ N(0,1))$ and the square root of the time increment. Another way to say this is that the Brownian increment comes from $rv ∼ N(0,t)$, where $t$ is the time increment. We obtain the Brownian path by taking the cumulative sum of the Brownian increments.

The SDE has a closed-form solution (only a few SDEs have it):
<center>$S(t)=S_0\exp\{(\mu-\frac{1}{2}\sigma^2)t+\sigma W_t\}$</center>

Here, $S_0=S(0)$ is the initial value of the process, which in this case is the initial price of a stock. The preceding equation presents the relationship compared to the initial stock price.

For simulations, we can use the following recursive formula:

<center>$S(t_{i+1})=S(t_i)\exp[(\mu-\frac{1}{2}\sigma^2)(t_{i+1}-t_i)+\sigma \sqrt{t_{i+1}-t_i}Z_{i+1}]$</center>

Here, $Z_i$ is a Standard Normal random variable and $i=0,...T-1$ is the time index. This specification is possible because the increments of W are independent and normally distributed.

GBM is a process that does not account for mean-reversion and time-dependent volatility. That is why it is often used for stocks and not for bond prices, which tend to display long-term reversion to the face value.

In this recipe, we use Monte Carlo methods and the Geometric Brownian Motion to simulate Microsoft's stock prices one month ahead.

### How to do it...

1. Import libraries:

In [ ]:
pip install yfinance;

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

2. Define parameters for downloading data:

In [ ]:
RISKY_ASSET = 'MSFT'
START_DATE = '2019-01-01'
END_DATE = '2019-07-31'

3. Download data from Yahoo Finance:

In [ ]:
df = yf.download(RISKY_ASSET, start=START_DATE,
                 end=END_DATE)

print(f'Downloaded {df.shape[0]} rows of data.')

4. Calculate daily returns:

In [ ]:
adj_close = df['Close']
returns = adj_close.pct_change().dropna()

ax = returns.plot()
ax.set_title(f'{RISKY_ASSET} returns: {START_DATE} - {END_DATE}',
             fontsize=16)

plt.tight_layout()
plt.show()

print(f'Average return:', round(100 * returns['MSFT'].mean(), 2), '%')

In [ ]:
returns.head()

5. Split data into the training and test sets:

In [ ]:
train = returns['2019-01-01':'2019-06-30']
test = returns['2019-07-01':'2019-07-31']

6. Specify the parameters of the simulation:

In [ ]:
adj_close.loc[train.index[-1]]

In [ ]:
T = len(test) # T: Forecasting horizon
N = len(test) # N: Number of time increments in the forecasting horizon
S_0 = adj_close['MSFT'].loc[train.index[-1]] # S_0: Initial price, the last observation from the training set
N_SIM = 100 # Number of simulated paths
mu = train['MSFT'].mean()
sigma = train['MSFT'].std()

In [ ]:
sigma

7. Define the function used for simulations:

In [ ]:
def simulate_gbm(s_0, mu, sigma, n_sims, T, N,
                 random_seed=42):
    '''
    Function used for simulating stock returns using Geometric Brownian Motion.

    Parameters
    ------------
    s_0 : float
        Initial stock price
    mu : float
        Drift coefficient
    sigma : float
        Diffusion coefficient
    n_sims : int
        Number of simulations paths
    dt : float
        Time increment, most commonly a day
    T : float
        Length of the forecast horizon, same unit as dt
    N : int
        Number of time increments in the forecast horizon
    random_seed : int
        Random seed for reproducibility

    Returns
    -----------
    S_t : np.ndarray
        Matrix (size: n_sims x (T+1)) containing the simulation results.
        Rows respresent sample paths, while columns point of time.
    '''
    # set seed for reproducible results
    np.random.seed(random_seed)

    # time increment
    dt = T/N
    # Brownian increments (the matrix of increments, each row describes one sample path)
    dW = np.random.normal(scale = np.sqrt(dt), size=(n_sims, N))
    # calculate the cumulative sum of the Brownian increments
    W = np.cumsum(dW, axis=1)

    # generate a sequence of N numbers evenly spaced from dt to T
    time_step = np.linspace(dt, T, N)

    # broadcast the array `time_step` to a new shape (n_sims dimensions and each with N elements)
    time_steps = np.broadcast_to(time_step, (n_sims, N))

    # calculate the stock price at time t using the closed-form formula
    S_t = s_0 * np.exp((mu - 0.5 * sigma**2) * time_steps
                       + sigma * W)
    # add the initial stock price as the first element in each demension of `S_t`
    S_t = np.insert(S_t, 0, s_0, axis=1)

    return S_t

Remarks:
* For reproducible results, use `np.random.seed` before simulating the paths.

Notes:
* Random Numbers in NumPy
    * NumPy offers the `random` module to work with random numbers.
    * `numpy.random.normal(loc=0.0, scale=1.0, size=None)`
        * Draw random samples from a normal (Gaussian) distribution.
        * `size`: *int or tuple of ints, optional*. Output shape.

In [ ]:
np.random.normal(size=(2, 3))

Notes:
* `numpy.broadcast_to(array, shape)`
    * Broadcast an array to a new shape.
    * Raises: ValueError - If the array is not compatible with the new shape according to NumPy’s broadcasting rules.

In [ ]:
a=np.arange(4)
print(a)
print(a.reshape(4,1)) # reshape() is a method for ndarray
np.broadcast_to(a,(6,4))

In [ ]:
a

In [ ]:
# not compatible according to NumPy’s broadcasting rules
#np.broadcast_to(a,(4,1))

In [ ]:
np.broadcast_to(a,(2,4))

Notes:
* `numpy.cumsum(a, axis=None)`
    * Return the cumulative sum of the elements along a given axis.

In [ ]:
a = np.array([[1,2,3], [4,5,6]])

# numpy.cumsum
print(f'{a}\n')
print(f'{np.cumsum(a)}\n')
print(f'{np.cumsum(a, axis=0)}\n') # sum over rows for each of the 3 columns
print(f'{np.cumsum(a, axis=1)}\n') # sum over columns for each of the 2 rows

Notes:
* `numpy.insert(arr, obj, values, axis=None)`
    * Insert values along the given axis before the given indices.
    * `arr` : *array_like*. Input array.
    * `obj` : *int, slice or sequence of ints*. Object that defines the index or indices before which `values` is inserted.
    * `axis` : *int, optional*. Axis along which to insert `values`.  If `axis` is None then `arr` is flattened first.

In [ ]:
# numpy.insert
# Axis parameter not passed. The input array is flattened before insertion.
print(f'{np.insert(a,3,[11,12])}\n')

# Axis parameter passed. The values array is broadcast to match input array.
#along axis 0
print(f'{np.insert(a,1,11,axis = 0)}\n')

#along axis 1
print(f'{np.insert(a,1,11,axis = 1)}\n')

8. Run the simulations:

In [ ]:
# use the function defined earlier
gbm_simulations = simulate_gbm(S_0, mu, sigma, N_SIM, T, N)
np.shape(gbm_simulations)

9. Plot simulation results:

In [ ]:
# prepare objects for plotting

# get the dates
last_train_date = train.index[-1].date() # select the last index in training set
first_test_date = test.index[0].date()
last_test_date = test.index[-1].date()

# add title
plot_title = (f'{RISKY_ASSET} Simulation '
              f'({first_test_date}:{last_test_date})')

# select the indices (DatetimeIndex) of adjusted prices in the testing set
selected_indices = adj_close[last_train_date:last_test_date].index

# extract the dates from the indices
index = [date.date() for date in selected_indices]

# use a DataFrame to hold `gbm_simulations` data for plotting
gbm_simulations_df = pd.DataFrame(np.transpose(gbm_simulations), index=index)
# transpose the marix to match with `index`  set the index

# plotting
ax = gbm_simulations_df.plot(alpha=0.2, legend=False) # `alpha=0.2`: make the lines transparent to help the two lines stand out

# add a line for the average value of all sample pathsb
line_1, = ax.plot(index, gbm_simulations_df.mean(axis=1),
                  color='red')
# add a line for actual stock price of Microsoft in the test set
line_2, = ax.plot(index, adj_close[last_train_date:last_test_date],
                  color='blue')
ax.set_title(plot_title, fontsize=16)
ax.legend((line_1, line_2), ('mean', 'actual'))

plt.tight_layout()
plt.show()

Notes:
* *list unpacking*
    * Unpack values from a list into variables.
    * `,` is used for argument unpacking.
    * `line_1, = ax.plot(index, gbm_simulations_df.mean(axis=1),color='red')`: `plot` returns a single-element `list`, which is unpacked into `line`.

In [ ]:
# unpacking
a=[1]
print(a,type(a))
a,=[1]
print(a,type(a))


# list unpacking
name = ['Tom', 'Alice', 'Jerry']
Tom = name[0]
Alice = name[1]
Jerry = name[2]
print(Tom, Alice, Jerry)
Tom, Alice, Jerry = name
print(Tom, Alice, Jerry)

In the above plot, we observe that the average value from the simulations exhibits a positive trend due to the positive drift term.

Bear in mind that this visualization is only feasible for a reasonable number of sample paths. In real-life cases, we want to use significantly more sample paths than 100, as usually the more sample paths, the more accurate/reliable the results are.

Summary:

1. In Steps 2 to 4, we downloaded Microsoft's stock prices and calculated simple returns. In the next step, we divided the data into the training and test sets. We calculated the average and standard deviation of the returns from the training set to obtain the drift (`mu`) and diffusion (`sigma`) coefficients, which we used later for simulations. Additionally, in Step 6, we defined the following parameters:

    * T: Forecasting horizon; in this case, the number of days in the test set.
    * N: Number of time increments in the forecasting horizon.
    * S_0: Initial price. For this simulation, we take the last observation from the training set.
    * N_SIM: Number of simulated paths.

    Monte Carlo simulations use a process called **discretization**. The idea is to approximate the continuous pricing of financial assets by splitting the considered time horizon into a large number of discrete intervals. That is why, except for considering the forecasting horizon, we also need to indicate the number of time increments to fit into the horizon.

2. Step 7 is where we defined the function for running the simulations. It is good practice to define a function/class for such a problem, as it will also come in handy in the following recipes. We started by defining the time increment (`dt`) and the Brownian increments (`dW`). In the matrix of increments (size: `n_sims x N`), each row describes one sample path. From there, we calculated the Brownian paths (`W`) by running a cumulative sum (`np.cumsum`) over the rows. Then, we created a matrix containing the time steps (`time_steps`). To do so, we created an array of evenly spaced values within an interval (the horizon of the simulation). For that, we used `np.linspace`. Afterward, we broadcasted the array to the intended shape using `np.broadcast_to`. We used the closed-form formula to calculate the stock price at each point in time. Finally, we inserted the initial value into the first position of each row.

    There was no explicit need to broadcast the vector containing time steps. It would have been done automatically to match the required dimensions (the dimension of `W`). However, in languages such as R, there is no automatic broadcasting. This also gives us more control over what we are doing and makes the code easier to debug.

    In the preceding steps, we can recognize the drift as `(mu - 0.5 * sigma ** 2) * time_steps` and the diffusion as `sigma * W`.

    While defining this function, we followed the vectorized approach. By doing so, we avoided writing any `for` loops, which would be inefficient in the case of large simulations.

3. In Step 8, we visualized the simulated sample paths. To do so, we transposed the data and converted it into a `pandas` DataFrame. We did the transposition so that we had one path per column, which simplifies using the `plot` method of `pandas` DataFrame. This can also be done using pure `matplotlib`. Aside from the main plot, we added two extra lines. The first one represents the average value of all sample paths at a given point in time. The second one is the actual stock price of Microsoft in the test set. To visualize the simulated stock prices, we chose `alpha=0.2` to make the lines transparent. By doing this, it is easier to see the two extra lines.

### There's more

There are some statistical methods that make working with Monte Carlo simulations easier (higher accuracy, faster computations). One of them is a variance reduction method called **antithetic variates**. In this approach, we try to reduce the variance of the estimator by introducing negative dependence between pairs of random draws. This translates into the following: when creating sample paths, for each $[\epsilon_1,...,\epsilon_t]$, we also take the antithetic values $[-\epsilon_1,...,-\epsilon_t]$.

$Var(\frac{X+Y}{2})=\frac{1}{4}(Var(X)+Var(Y)+2Cov(X,Y))$

The advantages of this approach are:
* Reduction (by half) of the number of Standard Normal samples to be drawn in order to generate N paths
* Reduction of the sample path variance, while at the same time, improving the accuracy

We implemented this approach in the `simulate_gbm` function. Additionally, we made the function shorter by putting the majority of the calculations into one line.

Before we implemented these changes, we timed the old version of the function:

In [ ]:
%timeit gbm_simulations = simulate_gbm(S_0, mu, sigma, N_SIM, T, N)

Notes:
* `%timeit` is an ipython magic function, which can be used to time a particular piece of code (A single execution statement, or a single method).

* If you wanted to see all of the magics you can use, you could simply type: `%lsmagic` to get a list of both line magics and cell magics.

The new function is defined as follows:

In [ ]:
def simulate_gbm(s_0, mu, sigma, n_sims, T, N, random_seed=42, antithetic_var=False):
    '''
    Function used for simulating stock returns using Geometric Brownian Motion.

    Parameters
    ------------
    s_0 : float
        Initial stock price
    mu : float
        Drift coefficient
    sigma : float
        Diffusion coefficient
    n_sims : int
        Number of simulations paths
    dt : float
        Time increment, most commonly a day
    T : float
        Length of the forecast horizon, same unit as dt
    N : int
        Number of time increments in the forecast horizon
    random_seed : int
        Random seed for reproducibility
    antithetic_var : bool
        Boolean whether to use antithetic variates approach to reduce variance

    Returns
    -----------
    S_t : np.ndarray
        Matrix (size: n_sims x (T+1)) containing the simulation results.
        Rows respresent sample paths, while columns point of time.
    '''

    np.random.seed(random_seed)

    # time increment
    dt = T/N

    # Brownian
    if antithetic_var:
        dW_ant = np.random.normal(scale = np.sqrt(dt),
                                  size=(int(n_sims/2), N + 1)) # we generate a more column as we will directly replace the first column with the initial value
        dW = np.concatenate((dW_ant, -dW_ant), axis=0) # join along column
    else:
        dW = np.random.normal(scale = np.sqrt(dt),
                              size=(n_sims, N + 1))

    # simulate the evolution of the process
    S_t = s_0 * np.exp(np.cumsum((mu - 0.5 * sigma ** 2) * dt + sigma * dW,
                                 axis=1))
    # add the initial price
    S_t[:, 0] = s_0

    return S_t

Notes:
* `numpy.concatenate((a1, a2, ...), axis=0)`
    * Join a sequence of arrays along an existing axis.
    * `a1, a2, …`: *sequence of array_like*. The arrays must have the same shape, except in the dimension corresponding to axis (the first, by default).
    * `axis`: *int, optional*. The axis along which the arrays will be joined. If axis is None, arrays are flattened before use. Default is 0.

In [ ]:
# np.concatenate
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])
print(f'{a}\n')
print(f'{b}\n')
print(f'{np.concatenate((a, b))}\n') # default: axis=0
print(f'{np.concatenate((a, b.transpose()),axis=1)}\n')
print(f'{np.concatenate((a, b.transpose()),axis=None)}\n')

In [ ]:
a = np.array([[1, 2, 3], [4, 5, 6]])
print(a)
np.sum(a, axis = 1)

First, we run the simulations without antithetic variables:

In [ ]:
%timeit gbm_simulations = simulate_gbm(S_0, mu, sigma, N_SIM, T, N)

Then, we run the simulations with them:

In [ ]:
%timeit gbm_simulations = simulate_gbm(S_0, mu, sigma, N_SIM, T, N, antithetic_var=True)

We succeeded in making the function faster. If you are interested in pure performance, these simulations can be further expedited using `Numba`, `Cython`, or `multiprocessing`.

Other possible variance reduction techniques include:

* Control variates
* Common random numbers

## Pricing European Options using Simulations

Options are a type of derivative instrument because their price is linked to the price of the underlying security, such as stock. Buying an options contract grants the right, but not the obligation, to buy or sell an underlying asset at a set price (known as a strike) on/before a certain date. The main reason for the popularity of options is because they hedge away exposure to an asset's price moving in an undesirable way.

A European call/put option gives us the right (but again, no obligation) to buy/sell a certain asset on a certain expiry date (commonly denoted as *T*).

Some popular methods of options' valuation:

* Using analytic formulas
* Binomial tree approach
* Finite differences
* Monte Carlo simulations

European options are an exception in the sense that there exist an analytical formula for their valuation, which is not the case for more advanced derivatives, such as American or Exotic options.

To price options using Monte Carlo simulations, we use risk-neutral valuation, under which the fair value of a derivative is the expected value of its future payoff(s). In other words, we assume that the option premium grows at the same rate as the risk-free rate, which we use for discounting to the present value. For each of the simulated paths, we calculate the option's payoff at maturity, take the average of all the paths, and discount it to the present value.

In this recipe, we show how to code the closed-form solution to the Black-Scholes model and then use the simulation approach. For simplicity, we use fictitious input data, but real-life data could be used analogically.

### How to do it...

1. Import the libraries:

In [ ]:
import numpy as np
from scipy.stats import norm
from chapter_6_utils import simulate_gbm

Remarks:
* Download `chapter_6_utils.py` here and put the file in your working directory.
https://github.com/PacktPublishing/Python-for-Finance-Cookbook/blob/master/Chapter%2006/chapter_6_utils.py

2. Define the parameters for the valuation:

In [ ]:
S_0 = 100
K = 100
r = 0.05
sigma = 0.50
T = 1 # 1 year
N = 252 # 252 days in a year
dt = T / N # time step
N_SIMS = 1000000 # number of simulations
discount_factor = np.exp(-r * T)

3. Define the function using the analytical solution:

In [ ]:
def black_scholes_analytical(S_0, K, T, r, sigma, type='call'):
    '''
    Function used for calculating the price of European options using the analytical form of the Black-Scholes model.

    Parameters
    ------------
    s_0 : float
        Initial stock price
    K : float
        Strike price
    T : float
        Time to maturity in years
    r : float
        Annualized risk-free rate
    sigma : float
        Standard deviation of the stock returns
    type : str
        Type of the option. Allowable: ['call', 'put']

    Returns
    -----------
    option_premium : float
        The premium on the option calculated using the Black-Scholes model
    '''

    d1 = (np.log(S_0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S_0 / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

    if type == 'call':
         # the formula for the price of a European call option
        val = (S_0 * norm.cdf(d1, 0, 1) - K * np.exp(-r * T) * norm.cdf(d2, 0, 1))
    elif type == 'put':
         # the formula for the price of a European put option
        val = (K * np.exp(-r * T) * norm.cdf(-d2, 0, 1) - S_0 * norm.cdf(-d1, 0, 1))
    else:
        raise ValueError('Wrong input for type!')

    return val

Notes:
* Raising an Exception
    * We can use raise to throw an exception if a condition occurs. The statement can be complemented with a custom exception.

In [ ]:
# raise a custom exception
# x = 10
# if x > 5:
#     raise Exception('x should not exceed 5. The value of x was: {}'.format(x))

4. Valuate the call option using the specified parameters:

In [ ]:
# calculate the benchmark for the Monte Carlo simulations
black_scholes_analytical(S_0=S_0, K=K, T=T, r=r, sigma=sigma, type='call')

5. Simulate the stock path using GBM:

In [ ]:
gbm_sims = simulate_gbm(s_0=S_0, mu=r, sigma=sigma,
                       n_sims=N_SIMS, T=T, N=N)

6. Calculate the option premium:

In [ ]:
# use the terminal value and take the average of the payoffs and discount it to present the value by using the discount factor
premium = discount_factor * np.mean(np.maximum(0, gbm_sims[:, -1] - K))
premium

The calculated option premium is 21.7562.

Here, we can see that the option premium that we calculated using Monte Carlo simulations is close to the one from a closed-form solution of the Black-Scholes model. To increase the accuracy of the simulation, we could increase the number of simulated paths (using the `n_sims` parameter).

Summary:

1. In Step 2, we defined the parameters that we used for this recipe:

    * S_0: Initial stock price
    * K: Strike price, that is, the one we can buy/sell for at maturity
    * r: Annual risk-free rate
    * sigma: Underlying stock volatility (annualized)
    * T: Time until maturity in years
    * N: Number of time increments for simulations
    * n_sims: Number of simulated sample paths
    * discount_factor: Discount factor, which is used to calculate the present value of the future payoff

2. In Step 3, we defined a function for calculating the option premium using the closed-form solution to the Black-Scholes model (for non-dividend-paying stocks). We used it in Step 4 to calculate the benchmark for the Monte Carlo simulations.

    The analytical solution to the call and put options is:<br>
<center>$C(S_t,t)=N(d_1)S_t-N(d_2)Ke^{-r(T-t)}$ <br>$P(S_t,t)=N(-d_2)Ke^{-r(T-t)}-N(-d_1)S_t$ <br>
$d_1=\frac{1}{\sigma\sqrt{T-t}} [ln(\frac{S_t}{K})+(r+\frac{\sigma^2}{2}){(T-t)}]$ <br>
$d_2=d_1-\sigma\sqrt{T-t}$</center>

    Here, $N()$ stands for the **cumulative distribution function (CDF)** of the Standard Normal distribution and $T - t$ is the time to maturity expressed in years. Equation 1 represents the formula for the price of a European call option, while equation 2 represents the price of the European put option. Informally, the two terms in equation 1 can be thought of as:

    * The current price of the stock, weighted by the probability of exercising the option to buy the stock ($N(d_1)$) – in other words, what we could receive.
    * The discounted price of exercising the option (strike), weighted by the probability of exercising the option ($N(d_2)$) – in other words, what we are going to pay.

3. In Step 5, we used the GBM simulation function from the previous recipe to obtain 1,000,000 possible paths of the underlying asset. To calculate the option premium, we only looked at the terminal values, and for each path, calculated the payoff as follows:

    * $max(S_T-K,0)$ for the call option
    * $max(K-S_T,0)$ for the put option

4. In Step 6, we took the average of the payoffs and discounted it to present the value by using the discount factor.

### There's more

In the previous steps, we showed you how to reuse the GBM simulation to calculate the
European call option premium. However, we can make the calculations faster, as in the case
of European options we are only interested in the terminal stock price. The intermediate
steps do not matter. That is why we only need to simulate the price at time $T$ and use these
values to calculate the expected payoff. We show how to do this by using an example of a
European put option with the same parameters as we used before.

In [ ]:
# calculating the option premium using the analytical formula
black_scholes_analytical(S_0=S_0, K=K, T=T, r=r, sigma=sigma, type='put')

In [ ]:
# define the modified simulation function
def european_option_simulation(S_0, K, T, r, sigma, n_sims, type='call', random_seed=42):
    '''
    Function used for calculating the price of European options using Monte Carlo simulations.

    Parameters
    ------------
    S_0 : float
        Initial stock price
    K : float
        Strike price
    T : float
        Time to maturity in years
    r : float
        Annualized risk-free rate
    sigma : float
        Standard deviation of the stock returns
    n_sims : int
        Number of paths to simulate
    type : str
        Type of the option. Allowable: ['call', 'put']
    random_seed : int
        Random seed for reproducibility

    Returns
    -----------
    option_premium : float
        The premium on the option calculated using Monte Carlo simulations
    '''
    # set seed
    np.random.seed(random_seed)
    rv = np.random.normal(0, 1, size=n_sims)
    # simulate the price at time 𝑇 using the closed-form formula
    S_T = S_0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * rv)

    # calculate the payoff
    if type == 'call':
        payoff = np.maximum(0, S_T - K)
    elif type == 'put':
        payoff = np.maximum(0, K - S_T)
    else:
        raise ValueError('Wrong input for type!')

    # take the average of the payoffs and discount it to present the value
    premium = np.mean(payoff) * np.exp(-r * T)
    return premium

In [ ]:
european_option_simulation(S_0, K, T, r, sigma, N_SIMS, type='put')

The two values are close to each other. Further increasing the
number of simulated paths should increase the accuracy of the valuation.

https://diposit.ub.edu/dspace/bitstream/2445/110072/2/memoria.pdf

https://s2pnd-matematika.fkip.unpatti.ac.id/wp-content/uploads/2019/03/Marek-Capinski-Peter-E.-Kopp-Measure-integral-and-probability-Springer-2004.pdf

## Pricing American Options with Least Squares Monte Carlo

In this recipe, we learn how to valuate American options. The key difference between European and American options is that the latter can be exercised at any time before and including the maturity date – basically, whenever the underlying asset's price moves favorably for the option holder.

This behavior introduces additional complexity to the valuation and there is no closed-form solution to this problem. When using Monte-Carlo simulations, we cannot only look at the terminal value on each sample path, as the option's exercise can happen anywhere along the path. That is why we need to employ a more sophisticated approach called **Least Squares Monte Carlo (LSMC)**, which was introduced by Longstaff and Schwartz (2001).

First of all, the time axis spanning $[0,T]$ is discretized into a finite number of equally spaced intervals and the early exercise can happen only at those particular time-steps. Effectively, the American option is approximated by a Bermudan one. For any time step $t$, the early exercise is performed in case the payoff from immediate exercise is larger than the continuation value.

This is expressed by the following formula:<br>
<center>$V_t(s)=max(h_t(s),C_t(s))$</center>
Here, $h_t(s)$ stands for the option's payoff (also called the option's inner value, calculated as in the case of European options) and $C_t(s)$ is the continuation value of the option, which is defined as :<br>
<center>$C_t(s)=E^Q_t[e^{-rdt}V_{t+dt}(S_{t+dt})|S_t=s]$</center>

Here, $r$ is the risk-free rate, $dt$ is the time increment, and $E^Q_t(...|S_t=s)$ is the risk-neutral expectation given the underlying price. The continuation value is basically the expected payoff from not exercising the option at a given time.

When using Monte Carlo simulations, we can define the continuation value for each path and time as $e^{-rdt}V_{t}+d_{t,i}$, where $i$ indicates the sample path. Using this value directly is not possible as this would imply perfect foresight. That is why the LSMC algorithm uses linear regression to estimate the expected continuation value. In the algorithm, we regress the discounted future values (obtained from keeping the option) onto a set of basis functions of the spot price (time $t$ price). The simplest way to approach this is to use an $x$-degree polynomial regression. Other options for the basis functions include Legendre, Hermite, Chebyshev, Gegenbauer, or Jacobi polynomials.

We iterate this algorithm backward (from time $T-1$ to $0$) and at the last step take the average discounted value as the option premium. The premium of a European option represents the lower bound to the American option's premium. The difference is usually called the early exercise premium.

### How to do it...

1. Import the libraries:

In [ ]:
import numpy as np
from chapter_6_utils import (simulate_gbm,
                             black_scholes_analytical,
                             lsmc_american_option)

2. Define the parameters:

In [ ]:
S_0 = 36 # Initial stock price
K = 40 # Strike price, that is, the one we can buy/sell for at maturity
r = 0.06 # Annual risk-free rate
sigma = 0.2 # Underlying stock volatility (annualized)
T = 1 # Time until maturity in years
N = 50 # Number of time increments for simulations
dt = T / N
N_SIMS = 10 ** 5 # Number of simulated sample paths
discount_factor = np.exp(-r * dt) #Discount factor, which is used to calculate the present value of the future payoff
OPTION_TYPE = 'put'
POLY_DEGREE = 5

3. Simulate the stock prices using GBM:

In [ ]:
gbm_sims = simulate_gbm(s_0=S_0, mu=r, sigma=sigma, n_sims=N_SIMS,
                        T=T, N=N)

4. Calculate the payoff matrix:

In [ ]:
payoff_matrix = np.maximum(K - gbm_sims, np.zeros_like(gbm_sims)) # compare K - S_T with 0

Notes:
* `numpy.zeros_like(a)`
    * Return an array of zeros with the same shape and type as a given array.
    * `a` : *array_like*. The shape and data-type of `a` define these same attributes of the returned array.

In [ ]:
example=([1,2],[3,4])
np.zeros_like(example)

5. Define the value matrix and fill in the last column (time T):

In [ ]:
# matrix of option values over time
value_matrix = np.zeros_like(payoff_matrix)  # define a matrix of zeros of the same size as the payoff matrix
value_matrix[:, -1] = payoff_matrix[:, -1] # fill the last column of the value matrix with the last column of the payoff matrix

In [ ]:
value_matrix.shape

In [ ]:
value_matrix[:,-1]

6. Iteratively calculate the continuation value and the value vector in the given time:

![flowchart](https://raw.githubusercontent.com/songssssss/notebook-repo/0b67215646f8916048ca2cfab0827bff81975b04/chp6%20flowchart.jpg)

In [ ]:
# iterate the algorithm backward (from time T−1 to 0)
for t in range(N - 1, 0 , -1):
    # regress the discounted future values (obtained from keeping the option) onto a set of basis functions of the spot price (time t price)
    regression = np.polyfit(gbm_sims[:, t],
                            value_matrix[:, t + 1] * discount_factor,
                            POLY_DEGREE)
    # estimate the expected continuation value (getting the fitted values from the regression)
    continuation_value = np.polyval(regression, gbm_sims[:, t])
    # compare the expected continuation value to the payoff to see if the option should be exercised
    value_matrix[:, t] = np.where(
        payoff_matrix[:, t] > continuation_value, # If the payoff was higher than the expected value from continuation
        payoff_matrix[:, t], #  we set the value to the payoff
        value_matrix[:, t + 1] * discount_factor  # Otherwise, we set it to the discounted one-step-ahead value
    )

Notes:
* `numpy.polyfit(x, y, deg)`
    * Least squares polynomial fit.
    * `x` : array_like, shape (M,). x-coordinates of the M sample points.
    * `y` : array_like, shape (M,). y-coordinates of the sample points.
    * `deg` : *int*. Degree of the fitting polynomial.
    * Returns a vector of coefficients `p` that minimises the squared error in the order `deg`, `deg-1`, ... `0`.


Remarks:
* It is also possible to use `scikit-learn` for the polynomial fit. To do so, you need to combine `LinearRegression` with `PolynomialFeatures`.


In [ ]:
# np.polyfit
egt = 1 # choose time t=1 as an example
X = gbm_sims[:, egt]
Y = value_matrix[:, egt + 1] * discount_factor

example1 = np.polyfit(X,
                      Y,
                      POLY_DEGREE)
example1

In [ ]:
value_matrix[:, egt + 1]

In [ ]:
X

In [ ]:
pip install sklearn

In [ ]:
# sklearn.linear_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

X_ = X.reshape(-1, 1)
polynomial_features = PolynomialFeatures(degree = POLY_DEGREE)
X_TRANS = polynomial_features.fit_transform(X_)

example2 = LinearRegression()
example2.fit(X_TRANS, Y).coef_

Notes:
* `numpy.array.reshape(shape)`
    * Returns an array containing the same data with a new shape.
    * The new shape should be compatible with the original shape. If an integer, then the result will be a 1-D array of that length. One shape dimension can be -1. In this case, the value is inferred from the length of the array and remaining dimensions.
    * See https://www.codingem.com/numpy-reshape-minus-one/#:~:text=By%20Artturi%20Jalli,elements%20to%20a%201D%20array.

In [ ]:
a = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
print(a.reshape(2, -1))
print(a.reshape(-1,1))

Notes:
* `numpy.polyval(p, x)`
    * Evaluate a polynomial at specific values.
    * If `p` is of length N, this function returns the value: ``p[0]*x**(N-1) + p[1]*x**(N-2) + ... + p[N-2]*x + p[N-1]``

In [ ]:
# np.polyval
np.polyval([3,2,1], 10) # 3*10**2+2*10**1+1*10**0

Notes:
* `numpy.where(condition[,x,y])`
    * `condition` : When True, yield `x`, otherwise yield `y`.
    * `x, y` : Values from which to choose.
    * Returns:
        * If both `x` and `y` are specified, the output array contains elements of `x` where condition is True, and elements from `y` elsewhere.
        * If only `condition` is given, return the tuple `condition.nonzero()`, the indices where condition is True.

In [ ]:
# np.where
example = np.array([[1, 2, 3], [4, 5, 6]])
print(example)

con=np.where(example<3) # only condition is given, return the indices where the condition is true
print(con)

print(example[con]) # select the elements that satisfy the condition
print(np.where(example<3,'<3','>=3')) # When True, yield '<3', otherwise yield '>=3'

# see also (in Chinese): https://blog.csdn.net/caihuanqia/article/details/105428512

7. Calculate the option premium:

In [ ]:
# take the average discounted t=1 value as the option premium
option_premium = np.mean(value_matrix[:, 1] * discount_factor)
print(f'The premium on the specified American {OPTION_TYPE} option is {option_premium:.3f}')

8. Calculate the premium of a European put with the same parameters:

In [ ]:
euput = black_scholes_analytical(S_0=S_0, K=K, T=T, r=r, sigma=sigma,
                         type='put')
print(f'The price of European put option with the same parameters is {OPTION_TYPE} option is {euput:.3f}')

9. As an extra check, calculate the prices of the American and European call options:

In [ ]:
european_call_price = black_scholes_analytical(S_0=S_0, K=K, T=T,
                                               r=r, sigma=sigma)
american_call_price = lsmc_american_option(S_0=S_0, K=K, T=T, N=N, r=r,
                                           sigma=sigma, n_sims=N_SIMS,
                                           option_type='call',
                                           poly_degree=POLY_DEGREE)
# the prices of the American and European call options are close
print(f"The price of the European call is {european_call_price:.3f}, and the American call's price (using {N_SIMS} simulations) is {american_call_price:.3f}")

Remarks:
* To make it easier, we put the entire algorithm for LSMC into one function `lsmc_american_option`, which is available in this book's GitHub repository `chapter_6_utils.py`.

Summary:

1. In Step 2, we once again defined the parameters of the considered American option. For comparison's sake, we took the same values that Longstaff and Schwartz (2001) did. In Step 3, we simulated the stock's evolution using the `simulate_gbm` function from the previous recipe. Afterward, we calculated the payoff matrix of the put option using the same formula that we used for the European options.

2. In Step 5, we prepared the matrix of option values over time, which we defined as a matrix of zeros of the same size as the payoff matrix. We filled the last column of the value matrix with the last column of the payoff matrix, as at the last step there are no further computations to carry out – the payoff is equal to the European option.

3. Step 6 is where we ran the backward part of the algorithm from time $T-1$ to $0$. At each of these steps, we estimated the expected continuation value as a cross-sectional linear regression. We fitted the $5^{th}$-degree polynomial to the data using `np.polyfit`. Then, we evaluated the polynomial at specific values (using `np.polyval`), which is the same as getting the fitted values from a linear regression. We compared the expected continuation value to the payoff to see if the option should be exercised. If the payoff was higher than the expected value from continuation, we set the value to the payoff. Otherwise, we set it to the discounted one-step-ahead value. We used `np.where` for this selection.

4. In Step 7 of the algorithm, we obtained the option premium by taking the average value of the discounted $t = 1$ value vector.

5. In the last two steps, we carried out some sanity checks regarding the implementation by calculating the option premiums of the European put and call options with the same parameters. For the call option, the premium on the American and European options should be equal, as it is never optimal to exercise the option when there are no dividends. Our results are very close, but we can obtain a more accurate price using more sample paths.

    In principle, the Longstaff-Schwartz algorithm should underprice American options because the approximation of the continuation value by the basis functions is just that, an approximation. As a consequence, the algorithm will not always make the correct decision about exercising the option. This, in turn, means that the option's value will be lower than in the case of the optimal exercise.

## Pricing American Options using Quantlib

In the previous recipe, we showed how to manually code the Longstaff-Schwartz algorithm. However, we can also use already existing frameworks for valuation of derivatives. One of the most popular ones is QuantLib. It is an open source C++ library that provides tools for the valuation of financial instruments. By using **Simplified Wrapper and Interface Generator (SWIG)**, it is possible to use QuantLib from Python (and some other programming languages, such as R or Julia). In this recipe, we show how to price the same American put option that we priced in the *Pricing American options with Least squares Monte Carlo* recipe, but the library itself has many more interesting features to explore.

### How to do it...

0: Defining parameters:

In [ ]:
S_0 = 36 # Initial stock price
r = 0.06 # Annual risk-free rate
sigma = 0.2 # Underlying stock volatility (annualized)
K = 40 # Strike price, that is, the one we can buy/sell for at maturity
OPTION_TYPE = 'put'
POLY_DEGREE = 5
R_SEED = 42 # set seed
N_SIMS = 10 ** 5  # Number of simulated sample paths
N = 50 # Number of time increments for simulations

1. Import the library:

In [ ]:
pip install QuantLib;

In [ ]:
import QuantLib as ql

Notes:

* `QuantLib`
    * The QuantLib project is aimed at providing a comprehensive software framework for quantitative finance. QuantLib is a free/open-source library for modeling, trading, and risk management in real-life. QuantLib offers tools that are useful both for practical implementation and for advanced modeling, with features such as market conventions, yield curve models, solvers, PDEs, Monte Carlo (low-discrepancy included), exotic options, VAR, and so on. For the documentation of QuantLib, please visit https://www.quantlib.org/reference/index.html

#### Quantlib Basics

##### Date Class

The `Date` object can be created using the constructor as `Date(day, month, year)`. It would be worthwhile to pay attention to the fact that `day` is the first argument, followed by `month` and then the `year`. This is different from the `python datetime` object instantiation.

In [ ]:
date = ql.Date(23, 7, 2021)
#date = ql.Date(1, 4, 2022)
print(date)

##### Calendar Class

The `Date` arithmetic above did not take holidays into account. But valuation of different securities would require taking into account the holidays observed in a specific exchange or country. The `Calendar` class implements this functionality for all the major exchanges.

In [ ]:
hk_calendar = ql.HongKong()
period = ql.Period(2, ql.Days)

print("Add 2 days in HK:", date + period)
print("Add 2 business days in HK:", hk_calendar.advance(date, period))

##### Interest Rate

The `InterestRate` class can be used to store the interest rate with the compounding type, day count and the frequency of compounding. Below we show how to create an interest rate of 6.0% compounded annually, using Actual/Actual day count convention.

In [ ]:
annual_rate = 0.06
day_count = ql.ActualActual(0)
  #`ActualActual()` Actual/Actual day count
compound_type = ql.Compounded
frequency = ql.Annual
interest_rate = ql.InterestRate(annual_rate, day_count, compound_type, frequency)

In [ ]:
t = 2.0
print(interest_rate.compoundFactor(t))
print((1+annual_rate)*(1.0+annual_rate))

##### Settings

In [ ]:
# today's date is returned if the evaluation date is set to the null date (its default value)
d = ql.Settings.instance().evaluationDate
print('Eval Date :', d)

# we can set it to a new value
ql.Settings.instance().evaluationDate = ql.Date(1, ql.January, 2020)
d = ql.Settings.instance().evaluationDate
print('New Eval Date :', d)

##### Instruments and pricing engines

Take a European option as a sample instrument. Building the option requires only the specification of its contract, so its payoff (it’s a call option with strike at 100) and its exercise, three months from today’s date. Market data will be selected and passed later, depending on the calculation methods.

In [ ]:
option = ql.EuropeanOption(ql.PlainVanillaPayoff(ql.Option.Call, 100.0),
                        ql.EuropeanExercise(ql.Date(3, ql.September, 2021)))

Take the analytic Black-Scholes formula as a sample pricing engine

First, we collect the quoted market data. We’ll assume flat risk-free rate and volatility, so they can be expressed by `SimpleQuote` instances. The underlying value is at 100, the risk-free value at 1%, and the volatility at 20%.

* `Quote` instances: those model numbers whose value can change and that can notify observers when this happens.

In [ ]:
u = ql.SimpleQuote(100.0)
r = ql.SimpleQuote(0.01)
sigma = ql.SimpleQuote(0.2)

In order to build the engine, the market data are encapsulated in a Black-Scholes process object.

Before that, we need to build flat curves for the risk-free rate and the volatility.

In [ ]:
riskFreeCurve = ql.FlatForward(0, hk_calendar, ql.QuoteHandle(r), day_count)
volatility = ql.BlackConstantVol(0, hk_calendar, ql.QuoteHandle(sigma), day_count)

We instantiate the process with the underlying value and the curves we just built.

* The `Handle` class is a smart pointer to pointer. The inputs are all stored into handles, so that we could change the quotes and curves used if we wanted.

In [ ]:
process = ql.BlackScholesProcess(ql.QuoteHandle(u),
                                 ql.YieldTermStructureHandle(riskFreeCurve),
                                 ql.BlackVolTermStructureHandle(volatility))

After having the process, we can finally use it to build the engine.

In [ ]:
engine = ql.AnalyticEuropeanEngine(process)

After having the engine, we can set it to the option and evaluate the latter.

In [ ]:
option.setPricingEngine(engine)
print(option.NPV())

##### Market changes
As mentioned before, market data are stored in `Quote` instances and thus can notify the option when any of them changes. We don’t have to do anything explicitly to tell the option to recalculate: once we set a new value to the underlying, we can simply ask the option for its NPV again and we’ll get the updated value.

In [ ]:
u.setValue(105.0)
r.setValue(0.006)
print(option.NPV())

2. Specify the calendar and the day counting convention:

In [ ]:
calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
day_counter = ql.ActualActual(0)

Remarks:
* The day counting convention determines the way interest accrues over time for various financial instruments, such as bonds. The `actual/actual` convention means that we use the actual number of elapsed days and the actual number of days in a year – 365 or 366. There are many other conventions such as `actual/365 fixed`, `actual/360`, and so on.

3. Specify the valuation date and the expiry date of the option:

In [ ]:
valuation_date = ql.Date(1, 1, 2018)
expiry_date =  ql.Date(1, 1, 2019)
ql.Settings.instance().evaluationDate = valuation_date

4. Define the option type (call/put), type of exercise and the payoff:

In [ ]:
if OPTION_TYPE == 'call':
    option_type_ql = ql.Option.Call
elif OPTION_TYPE == 'put':
    option_type_ql = ql.Option.Put

exercise = ql.AmericanExercise(valuation_date, expiry_date)
payoff = ql.PlainVanillaPayoff(option_type_ql, K)

5. Prepare the market-related data:

In [ ]:
# wrap the values in quotes so that the values can be changed and the changes are registered in the instrument
u = ql.SimpleQuote(S_0)
r = ql.SimpleQuote(0.05)
sigma = ql.SimpleQuote(0.5)

6. Specify the market-related curves:

In [ ]:
underlying = ql.QuoteHandle(u)
# volatility, which is constant as per our assumptions
volatility = ql.BlackConstantVol(0, calendar,
                                 ql.QuoteHandle(sigma),
                                 day_counter)

# the risk-free rate, which is also constant over time
risk_free_rate = ql.FlatForward(0, calendar,
                                ql.QuoteHandle(r),
                                day_counter)

In [ ]:
# volatility = ql.BlackConstantVol(valuation_date, calendar, sigma, day_counter)
# risk_free_rate = ql.FlatForward(valuation_date, r, day_counter)

# `TARGET` is a calendar that contains information on which days are holidays
# the price of the underlying instrument
underlying = ql.QuoteHandle(u)
# volatility, which is constant as per our assumptions
volatility = ql.BlackConstantVol(0, ql.TARGET(),
                                 ql.QuoteHandle(sigma),
                                 day_counter)

# the risk-free rate, which is also constant over time
risk_free_rate = ql.FlatForward(0, ql.TARGET(),
                                ql.QuoteHandle(r),
                                day_counter)

7. Plug in the market-related data into the BS process:

In [ ]:
bs_process = ql.BlackScholesProcess(
    underlying,
    ql.YieldTermStructureHandle(risk_free_rate),
    ql.BlackVolTermStructureHandle(volatility),
)

8. Instantiate the Monte Carlo engine for the American options:

In [ ]:
engine = ql.MCAmericanEngine(bs_process, 'PseudoRandom', timeSteps=N, # the number of time steps for discretization
                             polynomOrder=POLY_DEGREE,  # the degree/order of the polynomial in the LSMC algorithm
                             seedCalibration=R_SEED,
                             requiredSamples=N_SIMS) # the desired number of simulations

9. Instantiate the `option` object and set its pricing engine:

In [ ]:
option = ql.VanillaOption(payoff, exercise)
option.setPricingEngine(engine)

10. Calculate the option premium:

In [ ]:
option_premium_ql = option.NPV()

In [ ]:
print(f'The value of the American {OPTION_TYPE} option is: {option_premium_ql:.3f}')

Summary:
1. Since we wanted to compare the results we obtained here with those in the previous recipes, we used the same problem setup as we did there. For brevity, we will not look at all the code here, but we should run Step 2 from the previous recipe.
2. In Step 2, we specified the calendar and the day-counting convention.
3. In Step 3, we selected two dates – valuation and expiry – as we are interested in pricing an option that expires in a year. It is important to set `ql.Settings.instance().evaluationDate` to the considered evaluation date to make sure the calculations are performed correctly. In this case, the dates only determine the passage of time, meaning that the option expires within a year. We would get the same results (with some margin of error due to the random component of the simulations) using different dates with the same interval between them.

In [ ]:
# We can check the time to expiry (in years) by running the following code:
T = day_counter.yearFraction(valuation_date, expiry_date)
print(f'Time to expiry in years: {T}')
# Time to expiry in years: 1.0

4. Next, we defined the option type (call/put), the type of exercise (European, American, or Bermudan), and the payoff (Vanilla). In Step 5, we prepared the market data. We wrapped the values in quotes (`ql.SimpleQuote`) so that the values can be changed and the changes are registered in the instrument. This is important for calculating Greeks in the There's more section.

5. In Step 6, we defined the relevant curves. In this step, we specified the three important components of the **Black-Scholes (BS)** process, which are:
    * The price of the underlying instrument
    * Volatility, which is constant as per our assumptions
    * The risk-free rate, which is also constant over time
    
   We passed all these objects to the Black-Scholes process (`ql.BlackScholesProcess`), which we defined in Step 7. Then, we passed the process object into the special engine used for pricing American options using Monte Carlo simulations (there are many predefined engines for different types of options and pricing methods). At this point, we provided the desired number of simulations, the number of time steps for discretization, and the degree/order of the polynomial in the LSMC algorithm.

6. In Step 9, we created an instance of `ql.VanillaOption` by providing previously defined types of payoff and exercise. We also set the pricing engine using the `setPricingEngine` method.

7. Finally, we obtained the price of the option using the `NPV` method.

By doing this, we can see that the option premium we obtained using QuantLib is very similar to the one we calculated previously, which further validates our results.

### There's more

QuantLib also allows us to use variance reduction techniques such as antithetic values or **control variates**.

Now that we have completed the preceding steps, we can calculate Greeks. Greeks (from the letters of the Greek alphabet) represent the sensitivity of the price of derivatives (for example, the option premium) to a change in one of the underlying parameters (such as the price of the underlying asset, time to expiry, volatility, the interest rate, and so on). When there is an analytical formula available for the Greeks (when the QuantlLib engine is using analytical formulas), we could just access it by running, for example, `option.delta()`. However, in cases such as valuations using binomial trees or simulations, there is no analytical formula and we would receive an error (`RuntimeError: delta not provided`). This does not mean that it is impossible to calculate it, but we need to employ numerical differentiation and calculate it ourselves. In this example, we will only extract the delta. Therefore, the relevant two-sided formula is:

<center>$\Delta=\frac{P(S_0+h)-P(S_0-h)}{2h}$</center>

Here, *P(S)* is the price of the instrument given the underlying's price *S*; *h* is a very small increment.

In [ ]:
# calculate the delta
u_0 = u.value() # original value
h = 0.01

u.setValue(u_0 + h)
P_plus_h = option.NPV()

u.setValue(u_0 - h)
P_minus_h = option.NPV()

u.setValue(u_0) # set back to the original value

delta = (P_plus_h - P_minus_h) / (2 * h)

print(f'Delta of the option: {delta:.2f}')

The simplest interpretation of the delta is that the option's delta being equal to -1.25 indicates that, if the underlying stock increases in price by \\$1 per share, the option on it will decrease by \\$1.25 per share; otherwise, everything will be equal.

## Estimating Value-at-risk using Monte Carlo

**Value-at-risk** is a very important financial metric that measures the risk associated with a position, portfolio, and so on. It is commonly abbreviated to VaR, not to be confused with Vector Autoregression. VaR reports the worst expected loss – at a given level of confidence – over a certain horizon under normal market conditions. The easiest way to understand it is by looking at an example. Let's say that the 1-day 95% VaR of our portfolio is \\$100. This means that 95% of the time (under normal market conditions), we will not lose more than \\$100 by holding our portfolio over one day.

It is common to present the loss given by VaR as a positive (absolute) value. That is why in this example, a VaR of \\$100 means losing no more than \\$100.


There are several ways to calculate VaR, some of which are:

* Parametric Approach (Variance-Covariance)
* Historical Simulation Approach
* Monte Carlo simulations

In this recipe, we only consider the last method. We assume that we are holding a portfolio consisting of two assets (Facebook and Google) and that we want to calculate a 1-day value-at-risk.

### How to do it...

1. Import libraries:

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import seaborn as sns

In [ ]:
# set random seed for reproducibility
np.random.seed(42)

2. Define the parameters that will be used for this exercise:

In [ ]:
RISKY_ASSETS = ['GOOG', 'META']
SHARES = [5, 5] # the number of shares we have in our portfolio
START_DATE = '2018-01-01'
END_DATE = '2018-12-31'
T = 1
N_SIMS = 10 ** 5 # the number of simulations

3. Download data from Yahoo Finance:

In [ ]:
# downloaded the daily stock prices of Google and Facebook
df = yf.download(RISKY_ASSETS, start=START_DATE,
                 end=END_DATE)
print(f'Downloaded {df.shape[0]} rows of data.')

In [ ]:
df.head()

4. Calculate daily returns:

In [ ]:
# extract the adjusted close prices
adj_close = df['Close']
# converte adjusted close prices into simple returns
returns = adj_close.pct_change().dropna()
plot_title = f'{" vs. ".join(RISKY_ASSETS)} returns: {START_DATE} - {END_DATE}'
returns.plot(title=plot_title)

plt.tight_layout()
plt.show()

print(f'Correlation between returns: {returns.corr().values[0,1]:.2f}')

Notes:
* `string.join(iterable)`
    * The `join()` method joins all items in an iterable into a single string. Call this method on a string you want to use as a delimiter like comma, space etc.
    * `iterable`: any iterable (like list, tuple, dictionary etc.) whose items are strings
    * The method returns the string obtained by concatenating the items of an iterable.

In [ ]:
# example=[1,2] # items are ints
# "+".join(example)

In [ ]:
example=['1','2'] # items are strings
"+".join(example)

5. Calculate the covariance matrix:

In [ ]:
cov_mat = returns.cov()
cov_mat

Remarks:
* The Monte Carlo approach to determining the price of an asset employs random variables drawn from the Standard Normal distribution. For the case of calculating portfolio VaR, we need to account for the fact that the assets in our portfolio may be correlated.

6. Perform the Cholesky decomposition of the covariance matrix:

In [ ]:
chol_mat = np.linalg.cholesky(cov_mat)
chol_mat

Notes:
* `numpy.linalg.cholesky(a)`
    * Cholesky decomposition.
    * `a` : *(..., M, M) array_like*. Hermitian (symmetric if all elements are real), positive-definite input matrix.
    * Return the Cholesky decomposition, where `L` is lower-triangular.

In [ ]:
# np.linalg.cholesky
a = np.array([[4, 12, -16],
              [12, 37, -43],
              [-16, -43, 98]])
L=np.linalg.cholesky(a)
print(L)
print(np.matmul(L,L.T))

7. Draw correlated random numbers from Standard Normal distribution:

In [ ]:
rv = np.random.normal(size=(N_SIMS, len(RISKY_ASSETS)))
# multiply the resulting matrix by the matrix of random variables so that add correlation to the generated random variables
correlated_rv = np.transpose(np.matmul(chol_mat, np.transpose(rv)))

8. Define metrics used for simulations:

In [ ]:
r = np.mean(returns, axis=0).values # the historical averages of the asset return
sigma = np.std(returns, axis=0).values # the historical standard deviations of the asset return
S_0 = adj_close.values[-1, :] # the last known stock prices
P_0 = np.sum(SHARES * S_0) # the initial portfolio value

9. Calculate the terminal price of the considered stocks:

In [ ]:
# calculate possible 1-day-ahead stock prices for both assets
S_T = S_0 * np.exp((r - 0.5 * sigma ** 2) * T + # apply the analytical solution to the Geometric Brownian Motion SDE
                   sigma * np.sqrt(T) * correlated_rv)

10. Calculate the terminal portfolio value and calculate the portfolio returns:

In [ ]:
P_T = np.sum(SHARES * S_T, axis=1) # calculate the possible 1-day-ahead portfolio values
P_diff = P_T - P_0 # calculated the portfolio differences

11. Calculate VaR:

In [ ]:
P_diff_sorted = np.sort(P_diff) # sort the portfolio differences in ascending order
percentiles = [0.01, 0.1, 1.]
var = np.percentile(P_diff_sorted, percentiles)

for x, y in zip(percentiles, var):
    print(f'1-day VaR with {100-x}% confidence: {-y:.2f}$') # The X% VaR is simply the (1-X)-th percentile of the sorted portfolio differences

Notes:
* `numpy.sort(a, axis=- 1)`
    * Return a sorted copy of an array.
    * `axis` : *int or None, optional* Axis along which to sort. If None, the array is flattened before sorting. The default is -1, which sorts along the last axis.

In [ ]:
# np.sort
example = np.array([[1,3],[4,2]])
print(example)
print(np.sort(example))                # sort along the last axis (along columns)
print(np.sort(example, axis=None))     # sort the flattened array
print(np.sort(example, axis=0))        # sort along the first axis (along rows)

Notes:
* `numpy.percentile(a,q)`
    * Compute the q-th percentile of the data.
    * `a` : *array_like*. Input array or object that can be converted to an array.
    * `q` : *array_like of float*. Percentile or sequence of percentiles to compute, which must be between 0 and 100 inclusive.

In [ ]:
# percentile
example = range(1,100)
print(np.percentile(example, 50))
print(np.percentile(example, 0.1))

Notes:
* *Python Iterators*
    * An iterator is an object that contains a countable number of values.
    * An iterator is an object that can be iterated upon, meaning that you can traverse through all the values.
    * Lists, tuples, dictionaries, and sets are all iterable objects. They are iterable containers which you can get an iterator from. The built-in function `iter()` takes an iterable object and returns an iterator.
    * Each time we call the next method on the iterator gives us the next element. If there are no more elements, it raises a `StopIteration`.

In [ ]:
# astr = "Python"
# example = iter(astr)
# print(example)

# print(next(example))
# print(next(example))
# print(next(example))
# print(next(example))
# print(next(example))
# print(next(example))
# print(next(example)) # If there are no more elements, it raises a StopIteration

In [ ]:
# there are many functions which consume these iterables
print(",".join(["a", "b", "c"]))
print(",".join({"x": 1, "y": 2})) # join on dictionary: all dictionary keys are joined by default
print(list("python"))
print(list({"x": 1, "y": 2}))

Notes:
* `zip()`
    * The built-in function `zip()` aggregates the elements from multiple iterable objects (lists, tuples, etc.). It is used when iterating multiple list elements in a for loop.
    * By passing an iterable object (lists, tuples, etc.) as an argument of `zip()`, multiple elements can be obtained simultaneously in the for loop.
    * The result of the `zip()` function is an iterator. An iterator in Python is an object that contains a fixed number of elements and allows you to access each element in an ordered fashion (the next(iterator) function for an iterator). This is more efficient and more general-purpose — compared to creating a list and returning the list as a result. To fix this, you have to convert the iterator object in the iterable you want (e.g. set, list, tuple).
    * You can unzip zip object by using the asterisk operator `*`.
    * zip creates an object for iterating once over the results. This also means it's exhausted after one iteration. You need to call `zip(a,b)` every time you wish to use it or store the `list(zip(a,b))` result and use that repeatedly instead.

In [ ]:
# zip()
a = [1,2]
b = [3,4]
c = [3,4,5]

example1 = list(zip(a,b))
print(example1)

# using `*` to unzip
a1, b1 = zip(*zip(a,b))
print(a1,b1)

In [ ]:
# zip lists of different lengths
example2=zip(a,c)
print(example2) # zip object
print(tuple(example2)) # Python simply ignores the remaining elements of the longer list

In [ ]:
example3 = zip(a,b)
print(tuple(example3))
#it's exhausted after one iteration
print(example3)

In [ ]:
# display a readable version of the result
print(tuple(zip(a,b)))  # use the tuple() function
print([x for x in zip(a,b)]) # use list comprehension

12. Present the results on a graph:

In [ ]:
ax = sns.distplot(P_diff, kde=False) # `kde`: Whether to plot a gaussian kernel density estimate.
ax.set_title('''Distribution of possible 1-day changes in portfolio value
             1-day 99% VaR''', fontsize=16)
ax.axvline(var[2], 0, 10000) # Add a vertical line across the axes
plt.tight_layout()
plt.show()

Notes:
* Multiple Lines
    * Printing strings on multiple lines can make text more readable to humans. With multiple lines, strings can be grouped into clean and orderly text, formatted as a letter, or used to maintain the linebreaks of a poem or song lyrics.
    * To create strings that span multiple lines, triple single quotes `'''` or triple double quotes `"""` are used to enclose the string.

In [ ]:
print('''
Hello
World!
''')

The preceding plot shows the distribution of possible 1-day ahead portfolio values. We present the value-at-risk with the vertical line.

Summary:

1. In Steps 2 to 4, we downloaded the daily stock prices of Google and Facebook, extracted the adjusted close prices, and converted them into simple returns. We also defined a few parameters, such as the number of simulations and the number of shares we have in our portfolio.
    
    There are two ways to approach VaR calculations:<br>
    * **Calculate VaR from prices**: Using the number of shares and the asset prices, we can calculate the worth of the portfolio now and its possible value X days ahead.
    * **Calculate VaR from returns**: Using the percentage weights of each asset in the portfolio and the assets' expected returns, we can calculate the expected portfolio return X days ahead. Then, we can express VaR as the dollar amount based on that return and the current portfolio value.

2. The Monte Carlo approach to determining the price of an asset employs random variables drawn from the Standard Normal distribution. For the case of calculating portfolio VaR, we need to account for the fact that the assets in our portfolio may be correlated. To do so, in Steps 5 to 7, we calculated the historical covariance matrix, used the Cholesky decomposition on it, and multiplied the resulting matrix by the matrix of random variables. This way, we added correlation to the generated random variables.

    * Another possible option for making random variables correlated is to use the **Singular Value Decomposition (SVD)** instead of the Cholesky decomposition. The function we can use for this is `np.linalg.svd`.

3. In Step 8, we calculated metrics such as the historical averages of the asset return, the accompanying standard deviations, the last known stock prices, and the initial portfolio value. In Step 9, we applied the analytical solution to the Geometric Brownian Motion SDE and calculated possible 1-day-ahead stock prices for both assets.

4. To calculate the portfolio VaR, we calculated the possible 1-day-ahead portfolio values and the accompanying differences $(P_T-P_0)$ and sorted them in ascending order. The X% VaR is simply the (1-X)-th percentile of the sorted portfolio differences.

   * Banks frequently calculate the 1-day and 10-day VaR. To arrive at the latter, they can simulate the value of their assets over a 10-day interval using 1-day steps (discretization). However, they can also calculate the 1-day VaR and multiply it by the square root of 10. This might be beneficial for the bank if it leads to lower capital requirements.

### There's more

Calculating VaR using different approaches has its drawbacks, some of which are:

* Assuming a parametric distribution (variance-covariance approach).
* Not capturing enough tail risk.
* Not considering the so-called Black Swan events (unless they are already in the historical sample).
* Historical VaR can be slow to adapt to new market conditions.
* The Historical Simulation Approach assumes that past returns are sufficient to evaluate future risk (connects to the previous points).

Another general drawback of VaR is that it does not contain information about the size of the potential loss when it exceeds the threshold given by VaR. This is when **Expected Shortfall** (also known as conditional VaR or expected tail loss) comes into play. It simply states what the expected loss is in the worst X% of scenarios.

There are many ways to calculate the Expected Shortfall, but here, we present the one that is easily connected to the VaR and can be estimated using Monte Carlo.

Following on from the example of a two-asset portfolio, we would like to know the following: if the loss exceeds the VaR, how big will it be? To obtain that number, we need to filter out all losses that are higher than the value given by VaR and calculate their expected value by taking the average.

In [ ]:
var = np.percentile(P_diff_sorted, 5) # 95% VaR
expected_shortfall = P_diff_sorted[P_diff_sorted<=var].mean()

print(f'The 1-day 95% VaR is {-var:.2f}$, and the accompanying Expected Shortfall is {-expected_shortfall:.2f}$.')

Please bear in mind that, for Expected Shortfall, we only use a small fraction of all the simulations that were used to obtain the VaR. That is why, in order to have reasonable results for the Expected Shortfall, the overall sample must be large enough.

The 1-day 95% VaR is \\$4.48, while the accompanying Expected Shortfall is \\$5.27. We can interpret these results as follows: if the loss exceeds the 95% VaR, we can expect to lose \\$5.27 by holding our portfolio for 1 day.

References:<br>
https://www.interviewqs.com/blog/intro-monte-carlo<br>
https://pythonforfinance.net/2016/11/28/monte-carlo-simulation-in-python/#more-15561<br>
Price options using Monte Carlo Simulations<br>
http://www.codeandfinance.com/pricing-options-monte-carlo.html<br>
https://www.quantopia.net/quantlib-setting-up-quantlib-python-and-pricing-an-option/